In [27]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

# import os
# import django

# os.environ.setdefault("DJANGO_SETTINGS_MODULE", "T.settings")
# django.setup()

# from bf.models import InPlay




class BFSearch():

    def setUp(self):
        #         self.driver = webdriver.Firefox()
        PATH = r'C:\Users\Sales2\Desktop\kurs_every_day\T\chrom_driver\chromedriver.exe'
        # PATH = r'C:\Users\yablu\PycharmProjects\FootballParser\chrom_driver\chromedriver.exe'
        chrome_options = Options ()
        chrome_options.add_argument ("start-maximized")
        self.driver = webdriver.Chrome (PATH, options=chrome_options)
        # self.driver.get('https://www.betfair.com/exchange/plus/inplay/football')
        self.driver.wait = WebDriverWait (self.driver, 7)


    def tearDown(self):
        self.driver.quit ()

    def seven_elem(self, list_):
        """
        return  ['1', '0', 'FC Kuusysi', 'FC Futura', '93', '13']
        """
        try:
            if type (list_) == list:
                dell = list_.pop()[1::]
                dell = self.price_del_coma (dell)
                list_.append (dell)
                if str(list_[:1]) not in ['ПЕР', 'КОНЕЦ']:
                    dell = int (list_.pop (0)[:-1])
                    dell_2 = int (list_.pop (0)[1::])
                    sum_ = dell + dell_2
                    list_.append (sum_)
                else:
                    dell = list_.pop(0)
                    list_.append (dell)

                return list_  # ['1', '0', 'FC Kuusysi', 'FC Futura', '93', '13']
            else:
                print ('seven_list Error type')
        except IndexError:
            print('End list of elements')

    def six_elem(self, list_):
        """
        return  ['1', '0', 'FC Kuusysi', 'FC Futura', '93', '13']
        """
        try:
            if type (list_) == list:
                dell = list_.pop()[1::]
                dell = self.price_del_coma(dell)
                list_.append(dell)
                if list_[0] not in ['ПЕР', 'КОНЕЦ']: # ['PER','1', '0', 'FC Kuusysi', 'FC Futura', '93']
                    dell = list_.pop(0)[:-1]
                    list_.append(dell)
                else:
                    dell = list_.pop (0)
                    list_.append (dell)

                return list_  # ['1', '0', 'FC Kuusysi', 'FC Futura', '93', '13']
            else:
                print ('six_list Error type')
        except IndexError:
            print('End list of elements')

    def four_elem(self, list_):
        """
        return ['In-Play', 'PKKU/2', 'FC Kontu', '545']
        """
        try:
            if type (list_) == list:
                dell = list_.pop()[1::]
                dell = self.price_del_coma (dell)
                list_.append(dell)
                list_.append('-1')
                list_.reverse()
                list_.pop()
                list_.append ('-1')
                list_.append('-1')
                list_.reverse()
                return list_  # ['In-Play','In-Play', 'PKKU/2', 'FC Kontu', '545', 'In-Play']
            else:
                print('four_list Error type')
        except IndexError:
            print('End list of elements')

    def price_del_coma(self, elem:str):
        try:
            elem = elem.split(',')
            elem = ''.join(elem)
            elem = int(elem)
            return elem
        except Exception as e:
            print(e)

    def list_len(self, list_):
        try:
            # list_ = self.list_
            len_list = len(list_)
            if len_list == 4:
                return self.four_elem(list_)
            elif len_list == 6:
                return self.six_elem(list_)
            elif len_list == 7:
                return self.seven_elem(list_)
            else:
                print ('list_len > Error')
        except Exception as e:
            print (e)

            
    def find_coupon_table(self):
        try:
            self.res = self.driver.find_elements_by_class_name('coupon-table')
            return self.res
        except Exception as e:
            print ('not find_coupon_table', e)
        finally:
            print(self.res)
        
        
    def find_mod_link(self):
        try:
            if type(self.res) == list: 
                res = self.res[0]
                time.sleep(0.5)
                self.res = res.find_elements_by_class_name('mod-link')
                print('Ok, finded mod-link')
                print(self.res)
                return self.res
        
        except Exception as e:
            print ('not find_mod link', e)
   
    def find_text(self):
        if self.res:
            list_inplays = self.res[0]
            for i in list_inplays:
                print(i.text)

    def in_play(self):
        all_match_info = []
        if self.res:
            elem = self.res
            for i in elem:
                list_ = []

                text = i.text
                text = text.split('\n')
                info_list = self.list_len(text)
                list_.append(info_list)

                name_of_liga = i.get_attribute ('data-competition-or-venue-name')
                list_.append(name_of_liga)

                match_url = i.get_attribute ('href')
                list_.append(match_url)

                all_match_info.append(list_)
                self.all_match_info = all_match_info
        return self.all_match_info
    
    def to_dict(self):
        pass

    def append_to_db(self):
        model = InPlay()
        all_match_info = self.all_match_info
        for i in all_match_info:
            model.time_inplay = i[0][5]
            model.amaunt_match = i[0][4]
            model.runner_away = i[0][3]
            model.runner_home = i[0][2]
            model.scorre_away = i[0][1]
            model.scorre_home = i[0][0]
            model.football_liga = i[1]
            model.url_match = i[2]
            model.save()
            print(model.runner_home, 'add to base')

    def click_gb(self):
        self.driver.get('https://www.betfair.com/exchange/plus/inplay/football')
        time.sleep(3)
        
        self.driver.find_element_by_class_name('ssc-hls').click()
        self.driver.find_element_by_class_name('ssc-en_GB').click()
        time.sleep(5)



In [28]:
a = BFSearch()


In [29]:
a.setUp()
a.click_gb()

In [32]:

a.find_coupon_table()
a.find_mod_link()
a.in_play()


[<selenium.webdriver.remote.webelement.WebElement (session="04d00ba9c0d30d42e0ee54a7c958e3ff", element="188ae018-87d2-4b5e-ab11-c9eacaa344e3")>, <selenium.webdriver.remote.webelement.WebElement (session="04d00ba9c0d30d42e0ee54a7c958e3ff", element="7ba6868e-89cc-4e69-a797-371356f931cd")>]
Ok, finded mod-link
[<selenium.webdriver.remote.webelement.WebElement (session="04d00ba9c0d30d42e0ee54a7c958e3ff", element="fc5cfd57-dbc9-4aa9-9861-aa37dc9472ec")>, <selenium.webdriver.remote.webelement.WebElement (session="04d00ba9c0d30d42e0ee54a7c958e3ff", element="0ce71330-315d-4b38-ad31-cb318241b557")>, <selenium.webdriver.remote.webelement.WebElement (session="04d00ba9c0d30d42e0ee54a7c958e3ff", element="01d77dd8-f1f8-47fe-987c-01ecfebcfe1b")>, <selenium.webdriver.remote.webelement.WebElement (session="04d00ba9c0d30d42e0ee54a7c958e3ff", element="0252d88d-8d68-46cd-b695-d9d5d7535c2d")>, <selenium.webdriver.remote.webelement.WebElement (session="04d00ba9c0d30d42e0ee54a7c958e3ff", element="a9b2a5fc-fd

[[['1', '0', 'UOR-5 Youth', 'Arsenal Tula Youth', 5344, '45'],
  'russian-reserves',
  'https://www.betfair.com/exchange/plus/en/football/russian-reserves/uor-5-youth-v-arsenal-tula-youth-betting-30735037'],
 [['0', '0', 'Perak', 'Kuala Lumpur', 9013, '47'],
  'malaysian-super-league',
  'https://www.betfair.com/exchange/plus/en/football/malaysian-super-league/perak-v-kuala-lumpur-betting-30732799'],
 [['0', '0', 'Projek FAM-MSM', 'Selangor II', 3505, '46'],
  'malaysian-premier-league',
  'https://www.betfair.com/exchange/plus/en/football/malaysian-premier-league/projek-fam-msm-v-selangor-ii-betting-30732781'],
 [['1', '0', 'Gimnasia y Esgrim (Res)', 'Rosario Centra', 793, '45'],
  'argentinian-reserves',
  'https://www.betfair.com/exchange/plus/en/football/argentinian-reserves/gimnasia-y-esgrima-la-plata-res-v-rosario-centra-betting-30735146'],
 [['2', '0', 'CA Sarmiento (Res)', 'Atletico Platense (Res)', 2555, '48'],
  'argentinian-reserves',
  'https://www.betfair.com/exchange/plus

In [24]:
print(b)

[<selenium.webdriver.remote.webelement.WebElement (session="3b13cf6cdba6a3315c0cc35dce6b2410", element="71aee8af-3ce6-4b68-8627-7058d6942332")>, <selenium.webdriver.remote.webelement.WebElement (session="3b13cf6cdba6a3315c0cc35dce6b2410", element="c004e756-ff4c-40fe-8044-c3af0581bf4b")>, <selenium.webdriver.remote.webelement.WebElement (session="3b13cf6cdba6a3315c0cc35dce6b2410", element="7f71ee6f-4fb9-419b-a408-3179e6bae126")>, <selenium.webdriver.remote.webelement.WebElement (session="3b13cf6cdba6a3315c0cc35dce6b2410", element="ffef789b-2df0-4c46-bf5a-35d332363307")>, <selenium.webdriver.remote.webelement.WebElement (session="3b13cf6cdba6a3315c0cc35dce6b2410", element="ddddd631-2205-439e-a087-f9b9d933b232")>, <selenium.webdriver.remote.webelement.WebElement (session="3b13cf6cdba6a3315c0cc35dce6b2410", element="08bc4b3b-3538-431c-9fa4-d2b23d5344f4")>, <selenium.webdriver.remote.webelement.WebElement (session="3b13cf6cdba6a3315c0cc35dce6b2410", element="802247b5-e0c9-4aa5-b236-0ea8461a

In [25]:
print(len(b))
for i in b:
    print(i.text)

11


MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=59171): Max retries exceeded with url: /session/3b13cf6cdba6a3315c0cc35dce6b2410/element/71aee8af-3ce6-4b68-8627-7058d6942332/text (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000000006671240>: Failed to establish a new connection: [WinError 10061] Подключение не установлено, т.к. конечный компьютер отверг запрос на подключение',))

In [48]:
a.find_text()

TypeError: 'WebElement' object is not iterable

In [39]:
aa = a.driver.find_element_by_class_name('coupon-table')


In [40]:
bb = aa.find_elements_by_class_name('mod-link')

In [41]:
for i in bb:
    print(i.text)


In-Play
Qingdao Huanghai FC
Shenzhen FC
€0
In-Play
Chongqing Lifan
Guangzhou R&F
€88,913
90'
+3
2
1
G-Osaka
Oita
€121,502
HT
0
0
Canada (W)
Great Britain (W)
€67,555
HT
0
0
Chile (W)
Japan (W)
€83,940
HT
0
0
Krylia Sovetov Samara Youth
FC Nizhny Novgorod Y
€1,318
23'
1
0
Brazil (W)
Zambia (W)
€267,143
24'
1
0
Netherlands (W)
China (W)
€107,176
18'
1
0
Meizhou Hakka
Nantong Zhiyun F.C
€9,842
18'
0
0
Suzhou Dongwu
Kunshan
€9,797


In [42]:
print(aa.text)

In-Play
Matched
1 X 2
In-Play
Qingdao Huanghai FC
Shenzhen FC
€0
Closed
In-Play
Chongqing Lifan
Guangzhou R&F
€88,913
1.01
€2500
1000
€58
1000
€55
Suspended
90'
+3
2
1
G-Osaka
Oita
€121,502
15.5
€23
23
€82
1.08
€137
1.12
€371
21
€12
50
€16
HT
0
0
Canada (W)
Great Britain (W)
€67,555
5
€99
5.2
€52
2.34
€342
2.36
€191
2.62
€237
2.66
€13
HT
0
0
Chile (W)
Japan (W)
€83,940
16
€43
17.5
€108
3.95
€36
4.2
€47
1.44
€91
1.45
€120
HT
0
0
Krylia Sovetov Samara Youth
FC Nizhny Novgorod Y
€1,318
1.92
€113
1.98
€53
3.25
€57
3.4
€104
5.1
€36
5.5
€24
23'
1
0
Brazil (W)
Zambia (W)
€267,143
1.01
€30856
120
€13
260
€19
240
€12
800
€14
24'
1
0
Netherlands (W)
China (W)
€107,176
1.17
€354
1.19
€156
8
€98
9.4
€44
26
€30
27
€25
18'
1
0
Meizhou Hakka
Nantong Zhiyun F.C
€9,842
1.42
€628
1.43
€62
4.8
€114
5.1
€48
9.4
€18
10.5
€84
18'
0
0
Suzhou Dongwu
Kunshan
€9,797
3.75
€20
3.9
€126
3.15
€40
3.3
€152
2.3
€267
2.38
€16


In [24]:
aa[0]

TypeError: 'WebElement' object does not support indexing

In [22]:
a.tearDown()
